# `mlx-lm`

More info about using `mlx-lm` models with outlines [here](https://dottxt-ai.github.io/outlines/latest/reference/models/mlxlm/)

Table of Contents:
- [JSON Generation](#json-generation)
- [Text Generation](#text-generation)


## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/cookbook/extraction/

In [ ]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from mlx_lm.sample_utils import make_sampler
from pydantic import BaseModel

from outlines_haystack.generators.mlxlm import MLXLMJSONGenerator

In [ ]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [ ]:
prompt_template = """You are the owner of a pizza parlor. Customers \
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [ ]:
generator = MLXLMJSONGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    schema_object=Order,
)

### Standalone

In [ ]:
generator.warm_up()

In [ ]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt, generation_kwargs={"sampler": make_sampler(temp=0.5)})

### In a Pipeline

In [ ]:
pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMJSONGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        schema_object=Order,
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run(
    {
        "prompt_builder": {"order": "Is it possible to have 12 margheritas?"},
        "llm": {"generation_kwargs": {"sampler": make_sampler(temp=0.5)}},
    }
)

## Choice Generation

In [ ]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.mlxlm import MLXLMChoiceGenerator

In [ ]:
generator = MLXLMChoiceGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    choices=["Positive", "Negative"],
)

### Standalone

In [ ]:
generator.warm_up()

In [ ]:
generator.run(
    prompt="Classify the following statement: 'I love pizza'",
    generation_kwargs={"sampler": make_sampler(temp=0.5)},
)

### In a Pipeline

In [ ]:
prompt_template = "Classify the following statement: '{{statement}}'"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMChoiceGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        choices=["Positive", "Negative"],
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run(
    {
        "prompt_builder": {"statement": "I love Italian food"},
        "llm": {"generation_kwargs": {"sampler": make_sampler(temp=0.5)}},
    }
)

## Text Generation

In [ ]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.mlxlm import MLXLMTextGenerator

In [ ]:
generator = MLXLMTextGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
)

### Standalone

In [ ]:
generator.warm_up()

In [ ]:
generator.run(
    prompt="What is the capital of Italy?",
    generation_kwargs={"sampler": make_sampler(temp=0.5)},
)

### In a Pipeline

In [ ]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMTextGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run(
    {
        "prompt_builder": {"country": "France"},
        "llm": {"generation_kwargs": {"sampler": make_sampler(temp=0.5), "max_tokens": 20}},
    }
)